In [6]:
import pandas as pd
import numpy as np
import torch
import json

In [ ]:
# All of our fields have names, but we need them in numbers, so define a dictionary to convert.
global magDict
magDict = {
    'TOTUSJH': 0
    'TOTBSQ': 1
    'TOTPOT': 2
    'TOTUSJZ': 3
    'ABSNJZH': 4
    'SAVNCPP': 5
    'USFLUX': 6
    'TOTFZ': 7
    'MEANPOT': 8
    'EPSZ': 9
    'SHRGT45': 10
    'MEANSHR': 11
    'MEANGAM': 12
    'MEANGBT': 13
    'MEANGBZ': 14
    'MEANGBH': 15
    'MEANJZH': 16
    'TOTFY': 17
    'MEANJZD': 18
    'MEANALP': 19
    'TOTFX': 20
    'EPSY': 21
    'EPSX': 22
    'R_VALUE': 23
    'RBZ_VALUE': 24
    'RBT_VALUE': 25
    'RBP_VALUE': 26
    'FDIM': 27
    'BZ_FDIM': 28
    'BT_FDIM': 29
    'BP_FDIM': 30
    'PIL_LEN': 31
    'XR_MAX': 32
}

In [44]:
# Get the data from the JSON file, then return it as a tensor
def getTrainData(num, device, path="../data/"):
    # I might need to refactor these arguments to allow to get the test data. Problem for another day.
    # As it stands now, num is the number of the dataset, device is where to store it (CUDA), and path is the
    # path to the directory that holds the JSON files. 
    
    # Get the dictionary to assign names to numbers
    global magDict
    
    # Get the path to the file and open it
    path = '' + path + 'train_partition' + str(num) + '_data.json'
    file = open(path)
    
    # Declare a tensor to hold the data, and a list to hold the labels.
    # Dimensions: 0: length of the file. 1: the 33 fields in the data. 2: the 60 observations in each field. 
    # TODO: figure out the length of the file. 
    tnsr = torch.Tensor().new_empty((len(file), 33, 60), device=device)
    labels = []
    
    # Legacy
    train = []
    
#     # Track our place in the tensor
#     i=-1
    for line in file:
#         # Increment where we are
#         i += 1
        # Legacy
        if i > 2:
            break
        # Load the line as a dictionary. Row is an integer place and v is a smaller dictionary.
        d: dict = json.loads(line)
        for row, v in d.items():
            # append the label to our list
            labels.append(v['label'])
            # Legacy
            train.append(v['values'])
            
            # Break each individual dictionary into dictionaries of observations
            # Key is the string in magDict, and timeDict is a dictionary of observations over time
            for key, timeDict in v['values'].items():
                # Turn our name string into a numeric value
                location = magDict[key]
                # Get the measurements out of the time series dictionary
                for timeStamp, measurement in timeDict.items()
                    tnsr[int(row)][location][int(timeStamp)] = measurement
    # Close the file. I'm not a heathen
    file.close()
                    
    # print(train1[0])

    # All below here is legacy
    j = -1
    for k in train[0]:
        l=0
    #     print(len(train1[0].items()))
        j +=1
        print("'", k, "'", ': ', j, sep='')
    print(train[0]['EPSX'][f'{59}'])
    return tnsr, labels

train1 = getTrainData(1, 'cpu')

'TOTUSJH': 0
'TOTBSQ': 1
'TOTPOT': 2
'TOTUSJZ': 3
'ABSNJZH': 4
'SAVNCPP': 5
'USFLUX': 6
'TOTFZ': 7
'MEANPOT': 8
'EPSZ': 9
'SHRGT45': 10
'MEANSHR': 11
'MEANGAM': 12
'MEANGBT': 13
'MEANGBZ': 14
'MEANGBH': 15
'MEANJZH': 16
'TOTFY': 17
'MEANJZD': 18
'MEANALP': 19
'TOTFX': 20
'EPSY': 21
'EPSX': 22
'R_VALUE': 23
'RBZ_VALUE': 24
'RBT_VALUE': 25
'RBP_VALUE': 26
'FDIM': 27
'BZ_FDIM': 28
'BT_FDIM': 29
'BP_FDIM': 30
'PIL_LEN': 31
'XR_MAX': 32
0.09963967929642137
